https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("NY property data.csv")

In [3]:
data.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


### Levenshtein Distance

In [4]:
# conda install -c conda-forge python-levenshtein 

In [5]:
import Levenshtein as lev

In [6]:
findname = "Thomas Williams"
data["Levenshtein_Distance"] = data["OWNER"].apply(lambda x:lev.distance(str(x).lower(),findname.lower()))
data["Levenshtein_Ratio"] = data["OWNER"].apply(lambda x:lev.ratio(str(x).lower(),findname.lower()))

In [7]:
data["Levenshtein_Distance"].value_counts()

14    237348
13    223134
12    136602
15    127242
16     87673
17     83452
18     65190
11     47438
19     36640
10     11698
20      4737
9       3429
8       2660
7       2094
6       1073
5        353
21       144
4         65
3         11
2          7
1          3
0          1
Name: Levenshtein_Distance, dtype: int64

In [8]:
nameselect1 = data.loc[data.Levenshtein_Ratio >= 0.75,"OWNER"]
nameselect1.to_excel("nameselect1.xlsx")

In [9]:
lev.distance("Thomas WIlaalwifejlajflaeijfalijfliaaweflims3".lower(),"Thomas Williams".lower())

30

### FuzzyWuzzy

In [14]:
# conda install -c conda-forge fuzzywuzzy

Solving environment: done

## Package Plan ##

  environment location: /Users/apple/anaconda3

  added / updated specs:
    - fuzzywuzzy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fuzzywuzzy-0.17.0          |             py_0          18 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          18 KB

The following NEW packages will be INSTALLED:

  fuzzywuzzy         conda-forge/noarch::fuzzywuzzy-0.17.0-py_0



fuzzywuzzy-0.17.0    | 18 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [15]:
from fuzzywuzzy import fuzz

In [17]:
# Look at he shorter string, match and then calculate ratio
# seeks the score of the best matching length-k substring.
data["Partial_Score"] = data.OWNER.apply(lambda x: fuzz.partial_ratio(str(x).lower(),findname.lower()))

In [18]:
# Lowercase --> eliminate punctuation --> sort alphabetically --> concact ---> calculate ratio
# What if swtiched?
data["Token_Sort_Score"] = data.OWNER.apply(lambda x: fuzz.token_sort_ratio(str(x).lower(),findname.lower()))

In [19]:
# If the two character are very different in terns of length
# so after it lowercase --> eliminate punctuation --> sort alphabetically --> concact
# it then take the intersection of the two strings and then calculate the ratio
data["Token_Set_Score"] = data.OWNER.apply(lambda x: fuzz.token_set_ratio(str(x).lower(),findname.lower()))

In [20]:
names = data[["RECORD","OWNER","Levenshtein_Distance","Levenshtein_Ratio","Partial_Score"\
             ,"Token_Sort_Score","Token_Set_Score"]]

In [24]:
names = names.sort_values("Token_Set_Score",ascending = False)

In [26]:
nameoutput = names.head(500)

In [29]:
nameoutput.to_excel("nameoutput2.xlsx")